<a href="https://colab.research.google.com/github/adipurnamk/Belajar-Pengembangan-Machine-Learning/blob/master/Submission_2_Time_Series_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Dataset](https://www.kaggle.com/sumanthvrao/daily-climate-time-series-data)

Attribute Information:

Both hour.csv and day.csv have the following fields, except hr which is not available in day.csv

- instant: record index
- dteday : date
- season : season (1:winter, 2:spring, 3:summer, 4:fall)
- yr : year (0: 2011, 1:2012)
- mnth : month ( 1 to 12)
- hr : hour (0 to 23)
- holiday : weather day is holiday or not (extracted from [Web Link])
- weekday : day of the week
- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
+ weathersit :
- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- temp : Normalized temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-8, t_max=+39 (only in hourly scale)
- atemp: Normalized feeling temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-16, t_max=+50 (only in hourly scale)
- hum: Normalized humidity. The values are divided to 100 (max)
- windspeed: Normalized wind speed. The values are divided to 67 (max)
- casual: count of casual users
- registered: count of registered users
- cnt: count of total rental bikes including both casual and registered

In [136]:
import numpy as np
import pandas as pd
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import tensorflow as tf

In [137]:
!wget --no-check-certificate \
  https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip \
  -O /content/Bike-Sharing-Dataset.zip

--2020-09-21 16:28:59--  https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 279992 (273K) [application/x-httpd-php]
Saving to: ‘/content/Bike-Sharing-Dataset.zip’

/content/Bike-Shari 100%[===================>] 273.43K  1.06MB/s    in 0.3s    

2020-09-21 16:29:00 (1.06 MB/s) - ‘/content/Bike-Sharing-Dataset.zip’ saved [279992/279992]



In [138]:
import zipfile,os
local_zip = '/content/Bike-Sharing-Dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

os.listdir('/content')

['.config',
 'day.csv',
 'hour.csv',
 'Readme.txt',
 'Bike-Sharing-Dataset.zip',
 'sample_data']

In [139]:
data_hour = pd.read_csv('/content/hour.csv', index_col='dteday')
data_hour

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
dteday,,,,,,,,,,,,,,,,
2011-01-01,1,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
2011-01-01,2,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2011-01-01,3,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
2011-01-01,4,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
2011-01-01,5,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-12-31,17375,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642,11,108,119
2012-12-31,17376,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642,8,81,89
2012-12-31,17377,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,7,83,90


In [140]:
data_hour.isnull().sum()

instant       0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

In [141]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [142]:
data_hour.head()

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
dteday,,,,,,,,,,,,,,,,
2011-01-01,1,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
2011-01-01,2,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2011-01-01,3,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
2011-01-01,4,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
2011-01-01,5,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [143]:
data_hour = data_hour.drop(['instant', 'atemp', 'weekday', 'casual', 'registered'], axis=1)
data_hour

,season,yr,mnth,hr,holiday,workingday,weathersit,temp,hum,windspeed,cnt
dteday,,,,,,,,,,,
2011-01-01,1,0,1,0,0,0,1,0.24,0.81,0.0000,16
2011-01-01,1,0,1,1,0,0,1,0.22,0.80,0.0000,40
2011-01-01,1,0,1,2,0,0,1,0.22,0.80,0.0000,32
2011-01-01,1,0,1,3,0,0,1,0.24,0.75,0.0000,13
2011-01-01,1,0,1,4,0,0,1,0.24,0.75,0.0000,1
...,...,...,...,...,...,...,...,...,...,...,...
2012-12-31,1,1,12,19,0,1,2,0.26,0.60,0.1642,119
2012-12-31,1,1,12,20,0,1,2,0.26,0.60,0.1642,89
2012-12-31,1,1,12,21,0,1,1,0.26,0.60,0.1642,90


In [144]:
train_dataset = data_hour.sample(frac=0.8,random_state=0).astype(float)
test_dataset = data_hour.astype(float)

In [145]:
train_labels = train_dataset.pop('cnt').astype(float).values
test_labels = test_dataset.pop('cnt').astype(float).values

In [146]:
train_dataset = train_dataset.values[:, np.newaxis]
train_dataset.ndim

3

In [147]:
# train_set = windowed_dataset(train_labels, window_size=60, batch_size=100, shuffle_buffer=1000)
model = tf.keras.models.Sequential([
  tf.keras.layers.LSTM(60, return_sequences=True),
  tf.keras.layers.LSTM(60),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
])

In [148]:
optimizer = tf.keras.optimizers.SGD(lr=1.0000e-04, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=['mae', 'mse'])

In [149]:
history = model.fit(
    train_dataset,
    train_labels, 
    epochs=100, 
    batch_size=64, 
    validation_split=0.2,
    shuffle=False,
)

Epoch 1/100
174/174 [==============================] - 1s 7ms/step - loss: 188.5852 - mae: 189.0851 - mse: 68509.0156 - val_loss: 192.9114 - val_mae: 193.4108 - val_mse: 71715.2656
Epoch 2/100
174/174 [==============================] - 1s 3ms/step - loss: 188.1064 - mae: 188.6048 - mse: 68325.0391 - val_loss: 192.3687 - val_mae: 192.8655 - val_mse: 71500.9688
Epoch 3/100
174/174 [==============================] - 1s 3ms/step - loss: 187.4810 - mae: 187.9772 - mse: 68082.8672 - val_loss: 191.6260 - val_mae: 192.1234 - val_mse: 71205.5547
Epoch 4/100
174/174 [==============================] - 1s 3ms/step - loss: 186.5269 - mae: 187.0227 - mse: 67707.9531 - val_loss: 190.3730 - val_mae: 190.8690 - val_mse: 70697.8203
Epoch 5/100
174/174 [==============================] - 1s 3ms/step - loss: 184.7209 - mae: 185.2160 - mse: 66975.5234 - val_loss: 187.7614 - val_mae: 188.2569 - val_mse: 69591.6953
Epoch 6/100
174/174 [==============================] - 1s 3ms/step - loss: 180.3900 - mae: 180.